In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import os
import yaml
#from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

#from langchain_openai import OpenAI

from langchain.tools import tool
import requests

#telegram bot
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, MessageHandler, filters

In [2]:
#set fixed parameters 


### OPENAI API ###
with open('/home/christiane/git/stash/API/openai_api.yml', 'r') as file:
    OPENAI_API_KEY = yaml.safe_load(file)["api"]

### TELEGRAM API ###
with open('/home/christiane/git/stash/API/telegram_api.yml', 'r') as file:
    TELEGRAM_TOKEN = yaml.safe_load(file)["api"]
    
#check api key
#print(OPENAI_API_KEY)
#print(TELEGRAM_TOKEN)


### URL Coingecko ###
#url_coingecko = f"https://api.coingecko.com/api/v3/simple/price?ids={symbol.lower()}&vs_currencies=usd"
#print(url_coingecko)

In [18]:
# Initialize LLM

#OpenAI?

#temperature
# 0-0.3 -> focused/predictable
# 0.7-1.0-> creative mode

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [6]:
# Tool 1: Fetch current Crypto Price

@tool
def get_crypto_price_current(symbol: str) -> str:
    """Returns the current price of a given cryptocurrency symbol (e.g., BTC, ETH) using CoinGecko API"""
     #fix for user input BTC&bitcoin
    print(symbol)
    COIN_LOOKUP = {
        "btc": "bitcoin",
        "bitcoin": "bitcoin",
        "eth": "ethereum",
        "ethereum": "ethereum",
        "sol": "solana",
        "solana": "solana",
        "ada": "cardano",
        "cardano": "cardano",
        "doge": "dogecoin",
        "dogecoin": "dogecoin",
    }
    
    
    try:
        #for coingecko it has to be full name+lowercase
        coin_lowercase = symbol.strip().lower()   #symbol gets generated from package llm (extracted from input)
        coin_id = COIN_LOOKUP[coin_lowercase]     
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies=eur"
        response = requests.get(url).json()
        price = response[coin_id]["eur"]
        return f"The current price of {coin_id} is {price}€"
    
    except Exception as e:
        return f"Error fetching price: {e}"

    


In [55]:
# Tool 2 Fetch history max of cryptocurrency

@tool
def get_crypto_price_history(symbol: str) -> str:
    url = f"https://api.coingecko.com/api/v3/coins/{symbol.lower()}/market_chart?vs_currency=eur&days=max"
    try:
        response = requests.get(url).json()
        price = response[symbol.lower()]["eur"]

SyntaxError: incomplete input (3127424064.py, line 8)

In [1]:
import Mandelbrot


Mandelbrot.historical_data(crypto="BTC", curr="EUR", gran="d").head()


EUR
https://www.cryptodatadownload.com/cdd/Binance_BTCEUR_d.csv


,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume EUR,tradecount,Delta_High,Delta_Low
0,1750550400000,2025-06-22,BTCEUR,88728.21,89829.03,85411.45,87817.55,562.98370,4.913809e+07,75548,NaN,NaN
1,1750464000000,2025-06-21,BTCEUR,89755.10,90361.30,87663.31,88706.06,209.08262,1.861741e+07,32366,532.27,2251.86
2,1750377600000,2025-06-20,BTCEUR,90890.80,92480.78,88784.80,89755.09,390.23340,3.537340e+07,58041,2119.48,1121.49
3,1750291200000,2025-06-19,BTCEUR,91395.43,91750.00,90560.17,90889.81,143.58272,1.309223e+07,32390,-730.78,1775.37
4,1750204800000,2025-06-18,BTCEUR,91040.95,91749.99,90132.36,91390.86,287.99400,2.619193e+07,65908,-0.01,-427.81


In [8]:
# Agent Initialization 
tools = [get_crypto_price_current]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



/tmp/ipykernel_4783/687343348.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [18]:
# Telegram bot
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text
    response = agent.run(user_message)
    await update.message.reply_text(response)

app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

if __name__ == "__main__":
    print("Bot is running...")
    app.run_polling()

Bot is running...


RuntimeError: Cannot close a running event loop

In [10]:
# sample usage
if __name__ == "__main__":
    question = "What's the current price of ethereum and BTC?"
    print(agent.run(question))

    ## Example portfolio value
    #portfolio = {
   #     "bitcoin": 0.2,
    #    "ethereum": 1.5,
     #   "solana": 10
    #}
    #print(agent.run(f"Calculate the value of this portfolio: {portfolio}"))




> Entering new AgentExecutor chain...
 I should use the get_crypto_price_current function to get the current price of each cryptocurrency.
Action: get_crypto_price_current
Action Input: ETHETH

Observation: The current price of ethereum is 1967.33€
Thought: I should also get the current price of BTC.
Action: get_crypto_price_current
Action Input: BTCBTC

Observation: The current price of bitcoin is 88629€
Thought: I now know the final answer.
Final Answer: The current price of ethereum is 1967.33€ and the current price of bitcoin is 88629€.

> Finished chain.
The current price of ethereum is 1967.33€ and the current price of bitcoin is 88629€.


In [15]:
#test questions
agent.run("What's the price of BTC?")
#agent.run("Give me the price of bitcoin.")
#agent.run("eth price?")



> Entering new AgentExecutor chain...
 I should use the get_crypto_price_current function to get the current price of BTC.
Action: get_crypto_price_current
Action Input: "BTC"
Observation: The current price of bitcoin is 96070€
Thought: I now know the final answer
Final Answer: The current price of BTC is 96070€.

> Finished chain.


'The current price of BTC is 96070€.'

In [ ]:
@tool
def get_crypto_price_history(symbol: str) -> str:
    url = f"https://api.coingecko.com/api/v3/coins/{symbol.lower()}/market_chart?vs_currency=eur&days=max"
    try:
        response = requests.get(url).json()
        price = response[symbol.lower()]["eur"]

In [ ]:
##telegram connection

#Bot CryptoMOM
#CryptoMOMBot

#t.me/CryptoMOMBot
#token Http API